In [15]:
import os

In [16]:
# Check present working directory
%pwd

'c:\\Users\\deept\\ShopTalk'

In [17]:
# Change present working directory to thr root directory
os.chdir('../')

In [18]:
%pwd

'c:\\Users\\deept'

In [19]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [20]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
from pathlib import Path

In [21]:
class ConfigurationManager:
    CONFIG_FILE_PATH = r'C:\Users\deept\ShopTalk\config\config.yaml'
    PARAMS_FILE_PATH = r"C:\Users\deept\ShopTalk\params.yaml"
    SCHEMA_FILE_PATH = r"C:\Users\deept\ShopTalk\schema.yaml"
    def __init__(
        self,
        config_filepath = Path(CONFIG_FILE_PATH),
        params_filepath = Path(PARAMS_FILE_PATH),
        schema_filepath = Path(SCHEMA_FILE_PATH)):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [22]:
import os
import urllib.request as request
import zipfile
import tarfile
from mlProject import logger
from mlProject.utils.common import get_size


In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    
    
    def extract_tar_file(self):
        """
        Extracts a tar file into the data directory.
        
        Function returns None.
        """
        untar_path = self.config.untar_dir  # Assuming similar config attribute for directory
        os.makedirs(untar_path, exist_ok=True)  # Ensure the target directory exists
        
        # Using 'tarfile.open' to handle tar files (including compressed ones)
        with tarfile.open(self.config.local_data_file, 'r:*') as tar_ref:
            tar_ref.extractall(untar_path)

In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_tar_file()
except Exception as e:
    raise e

[2024-04-18 16:21:15,524: INFO: common: yaml file: C:\Users\deept\ShopTalk\config\config.yaml loaded successfully]
[2024-04-18 16:21:15,527: INFO: common: yaml file: C:\Users\deept\ShopTalk\params.yaml loaded successfully]
[2024-04-18 16:21:15,534: INFO: common: yaml file: C:\Users\deept\ShopTalk\schema.yaml loaded successfully]
[2024-04-18 16:21:15,536: INFO: common: created directory at: artifacts]
[2024-04-18 16:21:15,537: INFO: common: created directory at: artifacts/data_ingestion]


BoxKeyError: "'ConfigBox' object has no attribute 'unzip_dir'"